In [1]:
%load_ext autoreload
%autoreload 2


from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/w{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 25), 
        datetime.date(2020, 1, 20),
        datetime.timedelta(weeks=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/w{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 20), 
        datetime.date(2020, 2, 3),
        datetime.timedelta(weeks=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((283833, 36), (283833, 4), (75767, 36), (75767, 4))

In [4]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((36, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 34, 64)            256       
                                                                 
 flatten (Flatten)           (None, 2176)              0         
                                                                 
 dense (Dense)               (None, 8)                 17416     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/w-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])
model_view.save('../model/saved/w-view.keras')

Epoch 1/10
8870/8870 [==============================] - 17s 2ms/step - loss: 786.2346 - root_mean_squared_error: 28.0399 - val_loss: 1664.5541 - val_root_mean_squared_error: 40.7989
Epoch 2/10
8870/8870 [==============================] - 17s 2ms/step - loss: 357.9881 - root_mean_squared_error: 18.9206 - val_loss: 2107.8413 - val_root_mean_squared_error: 45.9112
Epoch 3/10
8870/8870 [==============================] - 16s 2ms/step - loss: 323.8746 - root_mean_squared_error: 17.9965 - val_loss: 2030.8599 - val_root_mean_squared_error: 45.0651
Epoch 4/10
8870/8870 [==============================] - 16s 2ms/step - loss: 327.1884 - root_mean_squared_error: 18.0883 - val_loss: 1065.2374 - val_root_mean_squared_error: 32.6380
Epoch 5/10
8870/8870 [==============================] - 16s 2ms/step - loss: 329.9218 - root_mean_squared_error: 18.1638 - val_loss: 1500.1304 - val_root_mean_squared_error: 38.7315
Epoch 6/10
8870/8870 [==============================] - 16s 2ms/step - loss: 294.8007 - ro

In [6]:

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/w-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])
model_cart.save('../model/saved/w-cart.keras')

Epoch 1/10
8870/8870 [==============================] - 18s 2ms/step - loss: 131.0128 - root_mean_squared_error: 11.4461 - val_loss: 800.3040 - val_root_mean_squared_error: 28.2896
Epoch 2/10
8870/8870 [==============================] - 18s 2ms/step - loss: 106.1104 - root_mean_squared_error: 10.3010 - val_loss: 850.5511 - val_root_mean_squared_error: 29.1642
Epoch 3/10
8870/8870 [==============================] - 17s 2ms/step - loss: 103.8851 - root_mean_squared_error: 10.1924 - val_loss: 835.6631 - val_root_mean_squared_error: 28.9078
Epoch 4/10
8870/8870 [==============================] - 17s 2ms/step - loss: 100.1175 - root_mean_squared_error: 10.0059 - val_loss: 827.5233 - val_root_mean_squared_error: 28.7667
Epoch 5/10
8870/8870 [==============================] - 17s 2ms/step - loss: 96.1083 - root_mean_squared_error: 9.8035 - val_loss: 781.0389 - val_root_mean_squared_error: 27.9471
Epoch 6/10
8870/8870 [==============================] - 17s 2ms/step - loss: 96.1116 - root_mean_

In [7]:

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/w-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])
model_remove_from_cart.save('../model/saved/w-remove_from_cart.keras')

Epoch 1/10
8870/8870 [==============================] - 17s 2ms/step - loss: 2568.5837 - root_mean_squared_error: 50.6812 - val_loss: 824.6478 - val_root_mean_squared_error: 28.7167
Epoch 2/10
8870/8870 [==============================] - 17s 2ms/step - loss: 32.9596 - root_mean_squared_error: 5.7410 - val_loss: 859.9853 - val_root_mean_squared_error: 29.3255
Epoch 3/10
8870/8870 [==============================] - 17s 2ms/step - loss: 30.6937 - root_mean_squared_error: 5.5402 - val_loss: 842.7704 - val_root_mean_squared_error: 29.0305
Epoch 4/10
8870/8870 [==============================] - 17s 2ms/step - loss: 29.2712 - root_mean_squared_error: 5.4103 - val_loss: 768.8834 - val_root_mean_squared_error: 27.7287
Epoch 5/10
8870/8870 [==============================] - 17s 2ms/step - loss: 30.7130 - root_mean_squared_error: 5.5419 - val_loss: 844.9459 - val_root_mean_squared_error: 29.0680
Epoch 6/10
8870/8870 [==============================] - 17s 2ms/step - loss: 29.4015 - root_mean_squar

In [8]:

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/w-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])
model_purchase.save('../model/saved/w-purchase.keras')

Epoch 1/10
8870/8870 [==============================] - 18s 2ms/step - loss: 28.6198 - root_mean_squared_error: 5.3497 - val_loss: 980.2920 - val_root_mean_squared_error: 31.3096
Epoch 2/10
8870/8870 [==============================] - 17s 2ms/step - loss: 7.1688 - root_mean_squared_error: 2.6775 - val_loss: 1021.5852 - val_root_mean_squared_error: 31.9622
Epoch 3/10
8870/8870 [==============================] - 17s 2ms/step - loss: 6.6706 - root_mean_squared_error: 2.5828 - val_loss: 1018.2416 - val_root_mean_squared_error: 31.9099
Epoch 4/10
8870/8870 [==============================] - 17s 2ms/step - loss: 6.3870 - root_mean_squared_error: 2.5273 - val_loss: 989.2228 - val_root_mean_squared_error: 31.4519
Epoch 5/10
8870/8870 [==============================] - 17s 2ms/step - loss: 6.2807 - root_mean_squared_error: 2.5061 - val_loss: 1008.7740 - val_root_mean_squared_error: 31.7612
Epoch 6/10
8870/8870 [==============================] - 17s 2ms/step - loss: 6.3355 - root_mean_squared_er